In [68]:
### Benjamin Tollison ###
import sympy as sp
from IPython.display import display, Math, Latex
from sympy.solvers.solveset import linsolve
from sympy.solvers.pde import pdsolve
from sympy import  lambdify, Matrix,sin,cos, sinh, cosh, exp, pi, symbols, integrate, Function, Eq
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)
def displayVariable(variable:str,RHS):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    display(Math(left + '=' + right))

In [106]:
gamma = 1.4
R = 287
T1 = 300
M1 = 3
T01 = T1*(1+((gamma-1)/2)*M1**2)
displayEquations(symbols('T_{0\,1}'),T01)
cp = (gamma*R)/(gamma-1)
cv = cp - R
h0 = cp*T01
displayEquations(symbols('h_0'),h0)
displayEquations(symbols('u_1'),-3*np.sqrt(gamma*R*T1))
a2 = np.sqrt(2.679)*np.sqrt(gamma*R*T1)
displayEquations(symbols('a_2'),a2)
u2 = -0.4752*a2
displayEquations(symbols('u_2'),u2)
w2 = u2 + 3*np.sqrt(gamma*R*T1)
displayEquations(symbols('w_2'),w2)
v_r = w2 + u2
displayEquations(symbols('v_r'),v_r)
T2 = 2.679*T1
displayVariable('T_2',T2)
M2 = v_r/np.sqrt(gamma*R*T2)
displayVariable('M_2',M2)
displayVariable('T_3',T01/3.888)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [70]:

def ThetaBetaMach(intial_theta_guess_rad,intial_beta_guess_rad,intial_mach_guess,theta_known:bool,beta_known:bool,mach_known:bool):
  theta,beta,M1,gamma = symbols('\\theta \\beta  M_1 \\gamma')
  eqn = 2*sp.cot(beta)*((M1**2*sin(beta)-1)/(M1**2*(gamma+cos(2*beta))+2)) - sp.tan(theta)
  partial_theta = eqn.diff(theta)
  partial_beta = eqn.diff(beta)
  partial_mach = eqn.diff(M1)
  f_prime = lambdify(args=[theta,beta,M1],expr=((partial_theta+partial_beta+partial_mach).subs(gamma,1.4)))
  f = lambdify(args=[theta,beta,M1],expr=(eqn.subs(gamma,1.4)))
  tolerance = 1e-8
  iteration_counter = 0
  iteration_table = {'theta_i+1':[],'beta_i+1':[],'M_i+1':[],'f_i+1':[]}
  if theta_known == False:
    theta_i = intial_theta_guess_rad
    while f(theta_i,intial_beta_guess_rad,intial_mach_guess) > tolerance or  f(theta_i,intial_beta_guess_rad,intial_mach_guess) < -tolerance:
      theta_i = theta_i - f(theta_i,intial_beta_guess_rad,intial_mach_guess)/f_prime(theta_i,intial_beta_guess_rad,intial_mach_guess)
      iteration_table['theta_i+1'].append(theta_i),iteration_table['beta_i+1'].append(intial_beta_guess_rad),iteration_table['M_i+1'].append(intial_mach_guess),iteration_table['f_i+1'].append(f(theta_i,intial_beta_guess_rad,intial_mach_guess))
      iteration_counter += 1
      if iteration_counter >=10000:
        return print('guess did not converge')
    displayEquations(symbols('\\theta'),theta_i)
    display(pd.DataFrame.from_dict(iteration_table))
    return theta_i
  if beta_known == False:
    beta_i = intial_beta_guess_rad
    while f(intial_theta_guess_rad,beta_i,intial_mach_guess) > tolerance or  f(intial_theta_guess_rad,beta_i,intial_mach_guess) < -tolerance:
      beta_i = beta_i - f(intial_theta_guess_rad,beta_i,intial_mach_guess)/f_prime(intial_beta_guess_rad,beta_i,intial_mach_guess)
      iteration_table['theta_i+1'].append(intial_theta_guess_rad),iteration_table['beta_i+1'].append(beta_i),iteration_table['M_i+1'].append(intial_mach_guess),iteration_table['f_i+1'].append(f(intial_theta_guess_rad,beta_i,intial_mach_guess))
      iteration_counter += 1
      if iteration_counter >=10000:
        return print('guess did not converge')
    displayEquations(symbols('\\beta'),beta_i)
    display(pd.DataFrame.from_dict(iteration_table))
    return beta_i
  if mach_known == False:
    mach_i = intial_mach_guess
    while f(intial_theta_guess_rad,intial_beta_guess_rad,mach_i) > tolerance or  f(intial_theta_guess_rad,intial_beta_guess_rad,mach_i) < -tolerance:
      mach_i = mach_i - f(intial_theta_guess_rad,intial_beta_guess_rad,mach_i)/f_prime(intial_theta_guess_rad,intial_beta_guess_rad,mach_i)
      iteration_table['theta_i+1'].append(intial_theta_guess_rad),iteration_table['beta_i+1'].append(intial_beta_guess_rad),iteration_table['M_i+1'].append(mach_i),iteration_table['f_i+1'].append(f(intial_theta_guess_rad,intial_beta_guess_rad,mach_i))
      iteration_counter += 1
      if iteration_counter >=10000:
        return print('guess did not converge')
    displayEquations(symbols('M_1'),mach_i)
    display(pd.DataFrame.from_dict(iteration_table))
    return mach_i

In [71]:
theta_rad = ThetaBetaMach(34*np.pi/180,np.pi/3,2.58199,False,True,True)
theta_degree = theta_rad*180/np.pi
displayEquations(symbols('\\theta'),theta_degree)
beta = np.pi/60
M = 2.58199
theta = np.arctan((2*(M**2*np.sin(beta)-1))/((M**2*(gamma+np.cos(2*beta))+2)*np.tan(beta)))
theta_degree = theta*180/np.pi
print(theta_degree)

<IPython.core.display.Math object>

,theta_i+1,beta_i+1,M_i+1,f_i+1
0,0.598250,1.047198,2.58199,7.425236e-03
1,0.600722,1.047198,2.58199,3.799183e-03
2,0.601984,1.047198,2.58199,1.942198e-03
3,0.602629,1.047198,2.58199,9.924404e-04
4,0.602959,1.047198,2.58199,5.070105e-04
5,0.603127,1.047198,2.58199,2.589877e-04
6,0.603213,1.047198,2.58199,1.322866e-04
7,0.603257,1.047198,2.58199,6.756772e-05
8,0.603279,1.047198,2.58199,3.451088e-05
9,0.603291,1.047198,2.58199,1.762664e-05


<IPython.core.display.Math object>

-54.13463628392


In [73]:
# Problem 6
M1n = np.sin(np.pi/3)*2.58199
displayEquations(symbols('M_{1n}'),M1n)
M2 = (0.5286**2+2.58199**2*np.cos(np.pi/3)**2)**.5
displayEquations(symbols('M_2'),M2)
T2 = 1.993*300
displayVariable('T_2',T2)
v2 = M2*np.sqrt(1.4*287*T2)
displayVariable('v_2',v2)
displayVariable('\\frac{A_1}{A_2}',5/2.75)
displayVariable('v_1',2.58199*np.sqrt(1.4*287*300))
v1 = 2.58199*np.sqrt(1.4*287*300)
displayVariable('\\frac{A_1}{A_2}',3.085*(v2/v1))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [84]:
# Problem 13
M1 = 1.7
M1n = M1*np.sin(59*np.pi/180)
displayVariable('M_{1n}',M1n)
w = M1*np.cos(59*np.pi/180)
M2n = 0.7157
M2 = np.sqrt(M2n**2 + w**2)
displayVariable('M_{2}',M2)
T2 = 1.255*282.31
displayVariable('T_2',T2)
v2 = M2*np.sqrt(1.4*287*T2)
displayVariable('v_2',v2)
P1 = 9.0971e4
P02 = 3.264*P1
displayVariable('P_{02}',P02*10**-6)
T02 = (1 + .2*M1**2)*282.31
displayVariable('T_{02}',T02-273.15)
displayVariable('F',P02*0.01465/4.448)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [98]:
# Problem 18
gamma = 1.4
M1 = 5
P1 = 2.65e4
P0 = (1+((gamma-1)/2)*M1**2)**(gamma/(gamma-1))*2.65e4
displayVariable('P_0',P0*10**-5)
P0 = 32.65*P1
displayVariable('P_0',P0*10**-5)
# beta_rad = ThetaBetaMach(20*np.pi/180,70*np.pi/180,5,True,False,True)
# beta_degree = beta_rad*180/np.pi
# displayVariable('\\beta',beta_degree)
beta = 30*np.pi/180
M1n = M1*np.sin(beta)
w = M1*np.cos(beta)
displayVariable('M_{1n}',M1n)
M2n = 0.513
M2 = np.sqrt(M2n**2 + w**2)
displayVariable('M_2',M2)
P2 = 7.125*2.65*10**4
displayVariable('P_2',P2)
P0 = 24.83*P2
displayVariable('P_0',P0*10**-5)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>